In [1]:
import re
import requests
import csv
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import lxml
from lxml import etree as et
import datetime

In [2]:
ua = UserAgent()
URL = 'https://hotline.ua/catalog/'
date = str(datetime.datetime.now())[:16]

In [3]:
response = requests.get(URL, headers={'User-Agent': ua.chrome})

In [4]:
soup = BeautifulSoup(response.text)

In [5]:
response2 = requests.get('https://hotline.ua/auto/', headers={'User-Agent': ua.chrome})
soup2 = BeautifulSoup(response2.text)

In [6]:
def find_img(name):
    for img in soup2.find(class_='tiles-list').find_all('img'):
        if name in img.get('title'):
            src = 'https://hotline.ua/' + img.get('data-src')
            return src

In [7]:
category_list = [['Авто и Мото', 'https://hotline.ua/auto/', 'None']]

for item in soup.find('div', {'class': 'accordion-css'}).find_all('div'):
    if item.get('class')[0] == "bold":
        
        bold_category_name = item.a.text
        bold_category_url = 'https://hotline.ua/catalog' + item.a.get('href')
        bolt_img = find_img(bold_category_name)
        
        category_list.append([bold_category_name, bold_category_url, 'Авто и Мото', bolt_img])
        
        subcategory_block = item.next_sibling.next_sibling.find_all('a')
        for link in subcategory_block:
            category_list.append([link.text, 'https://hotline.ua/catalog' + link.get('href'), bold_category_name, find_img(link.text)])
            
category_list

[['Авто и Мото', 'https://hotline.ua/auto/', 'None'],
 ['Автозапчасти',
  'https://hotline.ua/catalog/zapchasti/',
  'Авто и Мото',
  None],
 ['Все бренды',
  'https://hotline.ua/catalog/zapchasti/',
  'Автозапчасти',
  'https://hotline.ua//img/uploads/clients/5e6a48e6a6978.png'],
 ['Шины и диски',
  'https://hotline.ua/catalog/auto/shiny-i-diski/',
  'Авто и Мото',
  None],
 ['Автомобильные насосы, компрессоры',
  'https://hotline.ua/catalog/auto/nasosy-i-kompressory/',
  'Шины и диски',
  'https://hotline.ua//img/uploads/clients/261=5e7489de17e0d.jpg'],
 ['Автошины',
  'https://hotline.ua/catalog/auto/avtoshiny-i-motoshiny/',
  'Шины и диски',
  'https://hotline.ua//img/uploads/clients/5e67ec8f9ad6a.jpg'],
 ['Грузовые шины',
  'https://hotline.ua/catalog/auto/gruzovye-shiny/',
  'Шины и диски',
  'https://hotline.ua//img/uploads/clients/1181=5e7489deebdec.jpg'],
 ['Домкраты',
  'https://hotline.ua/catalog/auto/domkraty/',
  'Шины и диски',
  'https://hotline.ua//img/uploads/clients/5

In [8]:
xml = et.Element('xml', version="1.0", encoding="UTF-8")
tree = et.ElementTree(xml)

yml = et.Element('yml_catalog', date=date)
shop = et.SubElement(yml, 'shop')

categories = et.SubElement(shop, 'categories')

offers = et.SubElement(shop, 'offers')

In [9]:
for category in category_list:
    try:
        image = category[3]
    except:
        image = ''
    name = category[0]
    url = category[1]
    parent = category[2]
    category = et.SubElement(categories, 'category', hotline_url=str(url), parentCategory = parent, image = str(image))
    category.text = name

In [10]:
info = et.tostring(yml, pretty_print=True, xml_declaration=True, encoding="UTF-8",
                   doctype='<!DOCTYPE yml_catalog SYSTEM "shops.dtd">').decode("utf-8")

In [11]:
with open(r'C:\Users\Эпсилон\Desktop\hotline.xml', 'w', encoding='utf-8') as file:
    file.write(info)